In [1]:
fun recalculateVisits(tables: List<MutableList<Int>>, playersVisits: Map<Int, BooleanArray>) {
    tables.forEach {table ->
        for (i in table.indices) {
            val player = table[i]

            for (j in table.indices) {
                if (i == j) {
                    continue
                }

                playersVisits[player]!![table[j]] = true
            }
        }
    }
}

In [2]:
fun getFreeSeatInd(table: List<Int>): Int {
    for (i in table.indices) {
        if (table[i] == -1) {
            return i
        }
    }

    return -1
}

In [3]:
fun findBiggestPotentialTable(
    tables: List<MutableList<Int>>,
    playersVisits: Map<Int, BooleanArray>,
    playerNum: Int
): Int {
    val tableKnownPersons = IntArray(tables.size) { Int.MAX_VALUE }

    for (tableInd in tables.indices) {
        var knownCount = 0
        val table: List<Int> = tables[tableInd]

        if (!isTableHasFreeSeat(table)) {
            continue
        }

        table.forEach {
            if (it == -1) {
                return@forEach
            }

            if (playersVisits[it]!![playerNum]) {
                knownCount++
            }
        }

        tableKnownPersons[tableInd] = knownCount
    }

    var minKnownPersons = Int.MAX_VALUE
    var maxTableValueInd = -1

    for (tableInd in tableKnownPersons.indices) {
        if (tableKnownPersons[tableInd] < minKnownPersons) {
            minKnownPersons = tableKnownPersons[tableInd]
            maxTableValueInd = tableInd
        }
    }

    return maxTableValueInd
}

In [4]:
fun isTableHasFreeSeat(table: List<Int>): Boolean {
    return table.stream()
        .anyMatch { playerNum: Int -> playerNum == -1 }
}

In [5]:
fun findIntrovert(playersVisits: Map<Int, BooleanArray>): Int {
    var maxUnvisited = 0
    var player = -1

    for ((curPlayer) in playersVisits) {
        var countUnvisited = 0

        for (j in playersVisits[curPlayer]!!.indices) {
            if (!playersVisits[curPlayer]!![j]) {
                countUnvisited++
            }
        }

        if (countUnvisited > maxUnvisited) {
            maxUnvisited = countUnvisited
            player = curPlayer
        }
    }

    return player
}

In [6]:
fun seatPlayers(tables: List<MutableList<Int>>, playersVisits: Map<Int, BooleanArray>) {
    val playersQueue: MutableMap<Int, BooleanArray> = HashMap(playersVisits)

    while (playersQueue.isNotEmpty()) {
        val introvertNum = findIntrovert(playersQueue)

        playersQueue.remove(introvertNum)

        val bestTableInd = findBiggestPotentialTable(tables, playersVisits, introvertNum)
        val freeSeatInd = getFreeSeatInd(tables[bestTableInd])

        tables[bestTableInd][freeSeatInd] = introvertNum
    }

    recalculateVisits(tables, playersVisits)
}

In [7]:
fun printSchedule(schedule: List<List<MutableList<Int>>>) {
    for (day in schedule.indices) {
        println("day $day\n")

        for (table in schedule[day].indices) {
            print("table $table: ")

            for (i in schedule[day][table].indices) {
                print(schedule[day][table][i].toString() + " ")
            }
            println()
        }
        println()
    }
}

In [8]:
fun printPlayers(players: Map<Int, BooleanArray>) {
    for (i in 0 until players.size) {
        print("player $i visited: ")
        for (j in players[i]!!.indices) {
            if (players[i]!![j]) {
                print("$j ")
            }
        }
        println()
    }
}

In [9]:
fun main() {
    val playersCount = 8
    val days = 2
    val tablesSeatsCount = intArrayOf(3, 3, 2).sortedArrayDescending()

    val playersVisits: Map<Int, BooleanArray> = (0 until playersCount).associateWith { BooleanArray(playersCount) }


    val schedule = (0 until days).map {
        tablesSeatsCount.map {
            (0 until it).map { -1 }
                .toMutableList()
        }.toMutableList()
    }.toMutableList()

    for (day in 0 until days) {
        seatPlayers(schedule[day], playersVisits)
    }

    printSchedule(schedule)
    printPlayers(playersVisits)
}

In [10]:
main()

day 0

table 0: 0 1 2 
table 1: 3 4 5 
table 2: 6 7 

day 1

table 0: 6 0 3 
table 1: 7 1 4 
table 2: 2 5 

player 0 visited: 1 2 3 6 
player 1 visited: 0 2 4 7 
player 2 visited: 0 1 5 
player 3 visited: 0 4 5 6 
player 4 visited: 1 3 5 7 
player 5 visited: 2 3 4 
player 6 visited: 0 3 7 
player 7 visited: 1 4 6 
